##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/io/tutorials/genome"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/io/tutorials/genome.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/io/tutorials/genome.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
      <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/io/tutorials/genome.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Visão geral

Este tutorial demonstra o pacote `tfio.genome`, que conta com funcionalidade IO de genômica usada com frequência – especificamente, ler diversos formatos de arquivo de genômica e também fornecer algumas operações comuns para preparar os dados (por exemplo, codificação one-hot ou conversão de qualidade Phred em probabilidades).

Este pacote usa a biblioteca [Google Nucleus](https://github.com/google/nucleus) para fornecer parte da funcionalidade principal. 

## Configuração

In [ ]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
!pip install tensorflow-io

In [ ]:
import tensorflow_io as tfio
import tensorflow as tf

## Dados FASTQ

FASTQ é um formato de arquivo de genômica comum que armazena tanto as informações de sequência quanto as informações de qualidade base.

Primeiro, vamos baixar um arquivo `fastq` de amostra.

In [ ]:
# Download some sample data:
!curl -OL https://raw.githubusercontent.com/tensorflow/io/master/tests/test_genome/test.fastq

### Leitura de dados FASTQ

Agora, vamos usar `tfio.genome.read_fastq` para ler esse arquivo (é importante saber que uma API `tf.data` será lançada em breve).

In [ ]:
fastq_data = tfio.genome.read_fastq(filename="test.fastq")
print(fastq_data.sequences)
print(fastq_data.raw_quality)

Como podemos ver, os `fastq_data` retornados têm `fastq_data.sequences`, que é um tensor de strings contendo todas as sequências no arquivo fastq (e cada um pode ter um tamanho diferente) junto com `fastq_data.raw_quality`, que inclui as informações de qualidade codificadas em Phred sobre a qualidade de cada leitura base na sequência.

### Qualidade

Você pode usar uma operação helper para converter essas informações de qualidade em probabilidades, caso tenha interesse nisso.

In [ ]:
quality = tfio.genome.phred_sequences_to_probability(fastq_data.raw_quality)
print(quality.shape)
print(quality.row_lengths().numpy())
print(quality)

### Codificações one-hot

Talvez você também queira codificar os dados da sequência de genoma (que consistem em bases `A` `T` `C` `G`) usando um encoder one-hot. Uma operação integrada pode ajudar.


In [ ]:
one_hot = tfio.genome.sequences_to_onehot(fastq_data.sequences)
print(one_hot)
print(one_hot.shape)

In [ ]:
print(tfio.genome.sequences_to_onehot.__doc__)